- how to treat # and other punctuation
- Implement BPE
- Implement WordPiece
- Implement Spacy
- Implement Unigram
- Compare size of vocab
- Compare performance of model

In [1]:
import pandas as pd

In [2]:
# TASK: mit cleaned testen 

In [3]:
df_comments_bpe = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
df_comments_wp = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
df_comments_spacy = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
df_comments_unigram = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
df_comments = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
dataframes = [df_comments_bpe, df_comments_wp, df_comments_spacy, df_comments_unigram, df_comments]

for df in dataframes:
    df['cleaned'] = df['cleaned'].astype(str)

In [4]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors


In [5]:
# Initialize a tokenizer
tokenizer_bpe = Tokenizer(models.BPE())

# Use a pre-tokenizer to split text into words
tokenizer_bpe.pre_tokenizer = pre_tokenizers.Whitespace()

# Define a trainer
trainer = trainers.BpeTrainer(vocab_size=10000, min_frequency=2, special_tokens=["<unk>", "<s>", "</s>", "<pad>", "<mask>"])

# Train the tokenizer on the dataframe's text column
texts = df_comments_bpe["cleaned"].tolist()
tokenizer_bpe.train_from_iterator(texts, trainer)
vocab_bpe = tokenizer_bpe.get_vocab()
vocab_size_bpe = len(vocab_bpe)

In [6]:
# Tokenize the column and add tokens as a new column
df_comments_bpe['tokens'] = df_comments_bpe['cleaned'].apply(lambda x: tokenizer_bpe.encode(x).tokens)

# Display the dataframe with tokens
df_comments_bpe

,Unnamed: 0,id,body,cleaned,lemmatized,tokens
0,0,f1bk4k3,https://www.penmediainc.com/2019/09/23/climate...,nan,NaN,[nan]
1,1,frqfnw7,"Nice try, but false equivalency is still false...",nice try but false equivalency is still false ...,nice try but false equivalency be still false ...,"[nice, try, but, false, equivalency, is, still..."
2,2,dh9dp98,"I'm slightly torn on this, the idea of nations...",i slightly torn on this the idea of nations tr...,I slightly tear on this the idea of nation try...,"[i, slightly, torn, on, this, the, idea, of, n..."
3,3,i1vluk4,*Article contents as always:*\n\n&amp;#x200B;\...,article contents as always south australian pr...,article content as always south australian pre...,"[article, cont, ents, as, always, south, austr..."
4,4,fr9nzyu,What is the current outlook for ocean acidific...,what is the current outlook for ocean acidific...,what be the current outlook for ocean acidific...,"[what, is, the, current, out, look, for, ocean..."
...,...,...,...,...,...,...
1995,1995,h8859gb,The usual reason - their policies most closely...,the usual reason their policies most closely a...,the usual reason their policy most closely ali...,"[the, usual, reason, their, policies, most, cl..."
1996,1996,fqo3dlb,"it's not deep though, it is a surface level ob...",it not deep though it is a surface level obser...,it not deep though it be a surface level obser...,"[it, not, deep, though, it, is, a, surface, le..."
1997,1997,fhhct09,"**Hello u/UltimateWebRedditor, unfortunately y...",hello unfortunately your submission do blame c...,hello unfortunately your submission do blame c...,"[hello, unfortunately, your, submission, do, b..."
1998,1998,h4s5us0,&gt; New conventional nuclear plants... exorbi...,new conventional nuclear plants exorbitant cos...,new conventional nuclear plant exorbitant cost...,"[new, conventional, nuclear, plants, ex, orbit..."


## Word Piece

In [7]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, normalizers

# Initialize a tokenizer
tokenizer_wp = Tokenizer(models.WordPiece(unl_token="[UNK]"))

# Normalize the text (optional but recommended)
tokenizer_wp.normalizer = normalizers.BertNormalizer()

# Use a pre-tokenizer to split text into words
tokenizer_wp.pre_tokenizer = pre_tokenizers.Whitespace()

# Define a trainer
trainer = trainers.WordPieceTrainer(vocab_size=10000, min_frequency=2, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

# Train the tokenizer on the dataframe's text column
texts = df_comments_wp['cleaned'].tolist()
tokenizer_wp.train_from_iterator(texts, trainer)
vocab_wordpiece = tokenizer_wp.get_vocab()
vocab_size_wordpiece = len(vocab_wordpiece)

Ignored unknown kwargs option unl_token





In [8]:
# Apply the function to the dataframe
df_comments_wp['tokens'] = df_comments_wp['cleaned'].apply(lambda x: tokenizer_wp.encode(x).tokens)

# Display the dataframe with tokens
df_comments_wp

,Unnamed: 0,id,body,cleaned,lemmatized,tokens
0,0,f1bk4k3,https://www.penmediainc.com/2019/09/23/climate...,nan,NaN,[nan]
1,1,frqfnw7,"Nice try, but false equivalency is still false...",nice try but false equivalency is still false ...,nice try but false equivalency be still false ...,"[nice, try, but, false, equivalency, is, still..."
2,2,dh9dp98,"I'm slightly torn on this, the idea of nations...",i slightly torn on this the idea of nations tr...,I slightly tear on this the idea of nation try...,"[i, slightly, torn, on, this, the, idea, of, n..."
3,3,i1vluk4,*Article contents as always:*\n\n&amp;#x200B;\...,article contents as always south australian pr...,article content as always south australian pre...,"[article, content, ##s, as, always, south, aus..."
4,4,fr9nzyu,What is the current outlook for ocean acidific...,what is the current outlook for ocean acidific...,what be the current outlook for ocean acidific...,"[what, is, the, current, out, ##lo, ##ok, for,..."
...,...,...,...,...,...,...
1995,1995,h8859gb,The usual reason - their policies most closely...,the usual reason their policies most closely a...,the usual reason their policy most closely ali...,"[the, usual, reason, their, policies, most, cl..."
1996,1996,fqo3dlb,"it's not deep though, it is a surface level ob...",it not deep though it is a surface level obser...,it not deep though it be a surface level obser...,"[it, not, deep, though, it, is, a, surface, le..."
1997,1997,fhhct09,"**Hello u/UltimateWebRedditor, unfortunately y...",hello unfortunately your submission do blame c...,hello unfortunately your submission do blame c...,"[hello, unfortunately, your, submission, do, b..."
1998,1998,h4s5us0,&gt; New conventional nuclear plants... exorbi...,new conventional nuclear plants exorbitant cos...,new conventional nuclear plant exorbitant cost...,"[new, conventional, nuclear, plants, ex, ##or,..."


# Spacy

In [16]:
import spacy


In [18]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def tokenize_spacy(text):
    """
    Tokenizes the given text using the Spacy library.

    Parameters:
    text (str): The input text to be tokenized.

    Returns:
    list: A list of tokens extracted from the text, excluding punctuation and whitespace.
    """
    doc = nlp(text)
    return [token.text for token in doc if not token.is_punct and not token.is_space]

# Apply the tokenize function to the "cleaned" column
df_comments_spacy['tokens'] = df_comments_spacy['cleaned'].apply(tokenize_spacy)

# Build the vocabulary
# Flatten the list of tokens and create a set of unique tokens
vocab = set(token for tokens in df_comments_spacy['tokens'] for token in tokens)

# Calculate the length of the vocabulary
vocab_size_spacy= len(vocab)

In [ ]:
df_comments_spacy['tokens'] = df_comments_spacy['cleaned'].apply(tokenize_spacy)

## Unigram

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
# Initialize the CountVectorizer for unigram (default)
vectorizer = CountVectorizer()

# Fit the vectorizer on the "cleaned" column and transform the text to a term-document matrix
X = vectorizer.fit_transform(df_comments_unigram['cleaned'])

# Get the feature names (i.e., the vocabulary)
vocab = vectorizer.get_feature_names_out()

# Length of the vocabulary
vocab_size_unigram = len(vocab)


Comparison between the tokenizers

In [ ]:
# Collect the results
results = {
    "Tokenizer": ["BPE", "Unigram", "WordPiece", "spaCy"],
    "Vocab Size": [vocab_size_bpe, vocab_size_unigram, vocab_size_wordpiece, vocab_size_spacy]
}

# Create a DataFrame to display the results
df_results = pd.DataFrame(results)
print(df_results)

   Tokenizer  Vocab Size
0        BPE       10000
1    Unigram       14797
2  WordPiece       10000
3      spaCy       14818
